# Meeting minutes creator

We will create a minutes of meeting creator using OpenAI whisper(Open Source and available in Hugging Face) which will convert the Audio to text and Llama 3.2 which will prepare the Minutes of Meeting for us

### Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!


In [1]:
# Importing Libraries

import gdown
import os
import requests
import torch
import gc
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from transformers import pipeline

In [2]:
file_id = '1MONmc_mQ2ZG7rojUgLwkXn8uMJgVyg34'
url = f'https://drive.google.com/uc?export=download&id={file_id}'

output_path = '/home/alexender/Desktop/Projects/My_projects/Data/downloaded_audio.mp3' # audio file saving location
gdown.download(url, output_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1MONmc_mQ2ZG7rojUgLwkXn8uMJgVyg34
To: /home/alexender/Desktop/Projects/My_projects/Data/downloaded_audio.mp3
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14.4M/14.4M [00:02<00:00, 6.40MB/s]


'/home/alexender/Desktop/Projects/My_projects/Data/downloaded_audio.mp3'

In [3]:
#Loading HF Token

hf_token = os.getenv("HUGGING_FACE_WRITE_TOKEN")
login(hf_token)

# Open the file

audio_file = open(output_path, "rb")

In [4]:
audio_file

<_io.BufferedReader name='/home/alexender/Desktop/Projects/My_projects/Data/downloaded_audio.mp3'>

In [5]:
# Model Constants

LLAMA = "meta-llama/Llama-3.2-1B-Instruct"

### STEP 1: Transcribe Audio

#### Use Open Source for Transcription(OpenAI's Whisper models) - Hugging Face Pipelines

In [6]:
pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3-turbo",
    dtype=torch.float16,
    device='cuda',
    language='en',
    return_timestamps=True
)

result = pipe(output_path)
transcription = result["text"]
print(transcription)

Device set to use cuda


 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue, so that is the reason that the back of the logo is considered water, so let me see the creation of the logo here. yeah so that basically um kind of sums up the reason behind the logo and the um all the meanings behind the symbolism and uh i'll you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much, and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the 

In [7]:
del pipe
gc.collect()
torch.cuda.ipc_collect()
torch.cuda.empty_cache()

### STEP 2: Analyze & Report

In [8]:
# Now we will define system message, user prompt a

system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [9]:
# Adding Quantization config

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [10]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

# Applying Chat Templates

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

# Adding Streaming options

streamer = TextStreamer(tokenizer)

# Model

model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

# Generating output with streaming option

outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 Dec 2025

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue, so that is the reason that the back of the logo is considered water, so let me see the creation of the logo here. yeah so that basically um kind of sums up t

In [11]:
# Displaying the results in MarkDown

response = tokenizer.decode(outputs[0])
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 31 Dec 2025

You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now, and it's a very big issue, so that is the reason that the back of the logo is considered water, so let me see the creation of the logo here. yeah so that basically um kind of sums up the reason behind the logo and the um all the meanings behind the symbolism and uh i'll you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around denver so that we can kind of bring more people together and kind of share this whole idea of Indigenous Peoples Day. So thank you. Thank you so much, and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th. Please rise with the Pledge of Allegiance by Councilman Lopez. I pledge allegiance to the flag of the United States of America, to the republic for which it stands, one nation, under God, indivisible, with liberty and justice for all. All right, thank you, Councilman Lopez. Madam Secretary, roll call. Black? Clark? Here. Espinosa? Here. Flynn? Gilmore? Here. Here. Cashman? Here. Canich? Here. Lopez. Here. New. Here. Ortega. Here. Sussman. Here. Mr. President. Here. 11 present. 11 members present. We do have a quorum. Approval of the minutes. Are there any corrections to the minutes of October 2nd? Seeing none. Minutes of October 2nd stand approved. Council announcements. Are there any announcements by members of council? councilman clark thank you mr president um i just wanted to invite everyone down to the first ever halloween parade on broadway in lucky district seven it will happen on saturday october 21st at six o'clock p.m it'll move um along broadway from third to alameda it's going to be a fun family friendly event everyone's invited to come down wear a costume um there will be candy for the kids and um there are tiki zombies and uh 29 hearses and all kinds of fun and funky stuff on uh the fun and funky part of broadway so please join us october 21st at six o'clock for the broadway halloween bread thank you mr president all right thank you councilman clark i will be there all right uh presentations madam secretary do we have any presentations None, Mr. President. Communications. Do we have any communications? None, Mr. President. We do have one proclamation this evening, Proclamation 1127, an observance of the annual Indigenous Peoples Day in the city and county of Denver. Councilman Lopez, will you please read it? Thank you, Mr. President, with pride. Proclamation number 17, well, let me just say this differently. proclamation number 1127 series of 2017 and observance of the second annual indigenous people's day in the city and county of denver whereas the council of the city and county of denver recognizes that the indigenous peoples have lived and flourished on the lands known as the america since time immemorial and that denver and the surrounding communities are built upon the ancestral homelands of numerous indigenous tribes which include the southern ute the ute mountain ute tribes of colorado and whereas the tribal homelands and seasonal encampments of the Arapahoe and Cheyenne people along the banks of the Cherry Creek and South Platte River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City and whereas Colorado encompasses the ancestral homelands of 48 tribes and the city and county of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations and whereas on October 3rd, 2016, the City and County of Denver unanimously passed Council Bill 801 Series of 2016 officially designating the second Monday of October of each year as Indigenous Peoples Day in Denver, Colorado and whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to the community through Indigenous peoples' knowledge, science, philosophy, arts, and culture, and through these contributions, the City of Denver has developed and thrived. Whereas, the Indigenous community, especially youth, have made great efforts this year to draw attention to the contributions of Indigenous people, including Confluence Week, drawing record attendance to a National Indigenous Youth Leadership Conference leading conversations on inclusion with their peers and supporting increased Indigenous youth participation in science and engineering. Now therefore be it proclaimed by the Council of the City and County of Denver Section 1 that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of Indigenous people to our history, our past, our present and future and continues to promote the education of the Denver community about these historical and contemporary contributions of Indigenous people. Section 2, that the City and County of Denver, Colorado, does hereby observe October 9, 2017 as Indigenous Peoples Day. Section 3, that the Clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to this proclamation and that a copy be transmitted to the Denver American Indian Commission, the City and County of Denver School District No. 1, and the Colorado Commission on Indian Affairs. Thank you, Councilman Lopez. Your motion to adopt. Mr. President, I move that proclamation No. 1127, Series of 2017, be adopted. All right. It has been moved and seconded. It comes by members of Council. Councilman Lopez. Thank you, Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time, but as Indigenous Peoples Day in Denver officially for the second time. It's always awesome to be able to see not just this proclamation come by my desk, but to see so many different people from our community in our council chambers. um it was a very beautiful piece of artwork that you presented to us earlier and it is exactly the spirit that we drafted this proclamation and this actual the ordinance that created indigenous peoples day when we sat down and wrote it and as a community we couldn't think of anything else to begin except for the confluence of the two rivers and those confluence of the two rivers created such a great city and we live in such an amazing city and we we're all proud of it and sometimes we um and a lot of people from all over the country or all over the world are proud of it and sometimes a little too proud of it just tell them to go back home um but um i'm kidding when i say that um but the the really nice thing about this is that um we are celebrating indigenous people's day out of pride for who we are, who we are as a city, and the contributions of indigenous people to the city, not out of spite, not out of a replacement of one culture over the other, or out of contempt or disrespect. You know, I think of a quote that Cesar Chavez made very very popular, and it stuck with me for a very long time, and any time I have the opportunity, I speak in front of children, and especially children in our community that, you know, they often second-guess themselves on where they're coming from, who they are, and I always say that, you know, it's very important to be proud of who you're from, and the quote that I use from Cesar Chavez is, you know, pride in one's own culture does not require contempt or disrespect of another, right? And that's very important. It's very important for us to recognize that no matter who we are, where we come from in this society, that your pride in your own culture doesn't require, should not require the contempt or disrespect of another. And man, what a year to be for that to just sit on our shoulders for a while for us to think about right and so i wanted to um just to thank you all i think the commission um there's going to be a couple individuals that are going to come speak thank you for your art your lovely artwork for us to see uh what's in your heart and what now has become a probably is going to be a very important symbol uh for the community and also just for the work the daily work every single day we still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners right in poverty um without access to services right um without access to sobriety or even housing or jobs and what a what a what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores, right? So we have a lot of work to do. And these kind of proclamations and this day is not a day off. It's a day on in Denver, right, and addressing those critical issues. So I know that my colleagues are very supportive. I'm going to ask you to support this proclamation, as I know you always have done in the past. I'm very proud of today. Oh, and we made Time Magazine and Newsweek once again. today as being a leader in terms of the cities that are celebrating Indigenous Peoples Day. I wanted to make a point out of that. Thank you, Councilman Lopez, and thank you for sponsoring this. Councilman Ortega? Mr. President, I want to ask that my name be added. I don't think I could add much more to what Councilman Lopez has shared with us. I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state. I worked in the Lieutenant Governor's Office when the Commission on Indian Affairs was created and had the benefit of being able to go down to the Four Corners for a peace treaty signing ceremony between the Utes and the Comanches that had been sort of at odds with each other for about 100 years, and just being able to participate in that powwow was pretty awesome. And for those of you who continue to participate in the annual powwow, it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture. I mean, to see that the dance continues to be carried on, as well as the native language from generation to generation, is just so incredible because in so many cultures you know people have come here and assimilated to the you know the norms here and they lose their language and lose a lot of the culture and in the native community that hasn't happened that has you know commitment to just passing that on from generation to generation is is so important and so i'm i'm happy to be a co-sponsor of this tonight thank you all right thank you councilwoman ortega councilwoman canich thank you very much and i also want to thank my colleague for bringing this forward um and i i just wanted to to say a word to the artist about how beautiful and moving i thought this logo was and your description of it and i think one of the things um that is clear is you know the words sometimes don't convey the power of imagery or music or the other pieces that make up culture and so I think the art is so important and when you talked about water I was also thinking about land and I guess I just wanted to say thank you many of the Native American peoples of Colorado have been at the forefront or actually nationally of defending some of the public lands that have been protected over the last few years that are under attack right now and there are places that the communities have fought to protect but that everyone gets to enjoy and so i just think that it's an example of where cultural preservation intersects with environmental protection with you know recreation and all of the other ways that that public lands are so important and so i think i just wanted to say thank you for that because i think we have some very sacred places in our country that are at risk right now and so as we celebrate i appreciate that there's still a piece of resistance in here and i think that i just want to mention of solidarity and it mentioned a feeling of solidarity with that resistance so thank you and uh happy confluence week thank you uh councilman can each uh and seeing no other comments i'll just say a couple and um in a time of such divisive um ugliness and just despicable behavior from our leadership. The reason I'm so supportive of Indigenous Peoples Day is because it means inclusivity. It means respecting all, respecting those who have been silenced on purpose for a long time and whose history has not been told. And so we celebrate inclusivity in the face of such evil times, honestly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Summary:**

- The Denver City Council meeting was attended by 11 members.
- The meeting was held on Monday, October 9th.
- The Council introduced the creation of the logo for the Denver City Council.
- The logo is a representation of the confluence of two rivers, which symbolizes the city's connection to the indigenous peoples.
- The council members discussed the significance of the logo and the importance of celebrating Indigenous Peoples Day.
- The council adopted a proclamation, Proclamation 1127, which recognizes the contributions of indigenous peoples to the city and county of Denver.

**Discussion Points:**

- The council discussed the significance of the logo created by Madam Secretary.
- The council members expressed their pride in the city's connection to the indigenous peoples.
- The council members acknowledged the importance of celebrating Indigenous Peoples Day.

**Takeaways:**

- The council recognized the contributions of indigenous peoples to the city and county of Denver.
- The council adopted a proclamation, Proclamation 1127, which recognizes the contributions of indigenous peoples.
- The council members expressed their pride in the city's connection to the indigenous peoples.

**Action Items:**

- The council members:
  - Madam Secretary: presented the creation of the logo for the Denver City Council.
  - Councilman Lopez: read the proclamation, Proclamation 1127, and thanked the council members for their contributions.
  - Councilman Ortega: added his name to the proclamation.
  - Councilman Clark: expressed his support for Indigenous Peoples Day and acknowledged the importance of celebrating it.<|eot_id|>